In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from pathlib import Path

In [5]:
data = Path('/Users/jonniebarnsley/data')

measures = xr.open_dataset(data / 'MEaSUREs/MEaSUREs_antarctica_ice_velocity_v2_450m.nc')
bedmachine = xr.open_dataset(data / 'BedMachine' / 'BedMachineAntarctica-v3.nc')
bisicles = xr.open_dataset('/Users/jonniebarnsley/data/BedMachine/BedMachine_antarctica_v3_bisicles_1km.nc')

In [6]:
measures # MEaSUREs isn't big enough to fit BISICLES extent. Need to pad.

<xarray.Dataset> Size: 7GB
Dimensions:       (x: 12445, y: 12445)
Coordinates:
  * x             (x) float64 100kB -2.8e+06 -2.8e+06 ... 2.799e+06 2.8e+06
  * y             (y) float64 100kB 2.8e+06 2.8e+06 ... -2.799e+06 -2.8e+06
    lat           (y, x) float64 1GB ...
    lon           (y, x) float64 1GB ...
Data variables:
    coord_system  |S1 1B ...
    VX            (y, x) float32 620MB ...
    VY            (y, x) float32 620MB ...
    STDX          (y, x) float32 620MB ...
    STDY          (y, x) float32 620MB ...
    ERRX          (y, x) float32 620MB ...
    ERRY          (y, x) float32 620MB ...
    CNT           (y, x) int32 620MB ...
Attributes: (12/26)
    Conventions:               CF-1.6
    Metadata_Conventions:      CF-1.6, Unidata Dataset Discovery v1.0, GDS v2.0
    standard_name_vocabulary:  CF Standard Name Table (v22, 12 February 2013)
    id:                        vel_nsidc.CF16.nc
    title:                     MEaSURES Antarctica Ice Velocity Map 450m spacing
    product_version:            
    ...                        ...
    spatial_resolution:        450m
    time_coverage_start:       1995-01-01
    time_coverage_end:         2016-12-31
    project:                   NASA/MEaSUREs
    creator_name:              J. Mouginot
    license:                   No restrictions on access or use

In [7]:
# pad measures until it exceeds the extent of bisicles
N = 1000 # any big number will do
padded = measures.pad(x=(N, N), y=(N, N))

x = measures.x
dx = x[1] - x[0]
padded_axis_vals = np.arange(x.min()-N*dx, x.max()+N*dx+1, dx)
padded['x'] = padded_axis_vals
padded['y'] = -1*padded_axis_vals
padded

<xarray.Dataset> Size: 10GB
Dimensions:       (x: 14445, y: 14445)
Coordinates:
  * x             (x) float64 116kB -3.25e+06 -3.25e+06 ... 3.249e+06 3.25e+06
  * y             (y) float64 116kB 3.25e+06 3.25e+06 ... -3.249e+06 -3.25e+06
    lat           (y, x) float64 2GB nan nan nan nan nan ... nan nan nan nan nan
    lon           (y, x) float64 2GB nan nan nan nan nan ... nan nan nan nan nan
Data variables:
    coord_system  |S1 1B ...
    VX            (y, x) float32 835MB nan nan nan nan nan ... nan nan nan nan
    VY            (y, x) float32 835MB nan nan nan nan nan ... nan nan nan nan
    STDX          (y, x) float32 835MB nan nan nan nan nan ... nan nan nan nan
    STDY          (y, x) float32 835MB nan nan nan nan nan ... nan nan nan nan
    ERRX          (y, x) float32 835MB nan nan nan nan nan ... nan nan nan nan
    ERRY          (y, x) float32 835MB nan nan nan nan nan ... nan nan nan nan
    CNT           (y, x) float64 2GB nan nan nan nan nan ... nan nan nan nan nan
Attributes: (12/26)
    Conventions:               CF-1.6
    Metadata_Conventions:      CF-1.6, Unidata Dataset Discovery v1.0, GDS v2.0
    standard_name_vocabulary:  CF Standard Name Table (v22, 12 February 2013)
    id:                        vel_nsidc.CF16.nc
    title:                     MEaSURES Antarctica Ice Velocity Map 450m spacing
    product_version:            
    ...                        ...
    spatial_resolution:        450m
    time_coverage_start:       1995-01-01
    time_coverage_end:         2016-12-31
    project:                   NASA/MEaSUREs
    creator_name:              J. Mouginot
    license:                   No restrictions on access or use

In [8]:
# first convert bisicles into South Polar Stereographic coordinates by
# centering on the South Pole
x = bisicles.x
y = bisicles.y

centered = bisicles.copy()

centered['x'] = x - x.mean()
centered['y'] = y - y.mean()

centered

<xarray.Dataset> Size: 2GB
Dimensions:        (y: 6144, x: 6144)
Coordinates:
  * x              (x) float64 49kB -3.072e+06 -3.07e+06 ... 3.07e+06 3.072e+06
  * y              (y) float64 49kB -3.072e+06 -3.07e+06 ... 3.07e+06 3.072e+06
Data variables:
    mapping        |S1 1B ...
    mask           (y, x) float64 302MB ...
    firn           (y, x) float32 151MB ...
    surface        (y, x) float32 151MB ...
    thickness      (y, x) float32 151MB ...
    bed            (y, x) float32 151MB ...
    errbed         (y, x) float32 151MB ...
    source         (y, x) float64 302MB ...
    dataid         (y, x) float64 302MB ...
    geoid          (y, x) float64 302MB ...
    corrected_bed  (y, x) float32 151MB ...
Attributes: (12/17)
    Conventions:                 CF-1.7
    Title:                       BedMachine Antarctica
    Author:                      Mathieu Morlighem
    version:                     03-Jun-2022 (v3.4)
    nx:                          13333.0
    ny:                          13333.0
    ...                          ...
    ymax:                        3333000
    spacing:                     500
    no_data:                     -9999.0
    license:                     No restrictions on access or use
    Data_citation:               Morlighem M. et al., (2019), Deep glacial tr...
    Notes:                       Data processed at the Department of Earth Sy...

In [9]:
measures_1km = padded.interp(x=centered.x, y=centered.y, method='nearest')